In [2]:
import json
import os
import nest_asyncio

nest_asyncio.apply()

def get_config_data():
    with open("local_data.json") as json_file:
        data = json.load(json_file)
        return data["OPENAI_KEY"]["key"]
        
OPENAI_API_KEY = get_config_data()

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

# Use ollama in JSON mode
Settings.llm = OpenAI(
    model="gpt-4o-mini"
)
Settings.embed_model = OpenAIEmbedding(model_name="text-embedding-3-small")



In [6]:
llm = OpenAI(model="gpt-4o-mini")

## Define tools
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent


# Tool to accept user input to retrieve data from GEO using existing functions and ARCHS4PY
from llama_index.core.tools import FunctionTool
from pydantic import Field


def get_weather(
    location: str = Field(
        description="A city name and state, formatted like '<name>, <state>'"
    ),
) -> str:
    """Useful for getting the weather for a given location."""
    return f"The weather in {location} is sunny."


def get_time(
    location: str = Field(
        description="A city name and state, formatted like '<name>, <state>'"
    ),
) -> str:
    """Useful for getting the current time for a given location."""
    return f"The current time in {location} is 12:00 PM."


weather_tool = FunctionTool.from_defaults(get_weather, name="get_weather", description="Useful for getting the weather for a given location.")
time_tool = FunctionTool.from_defaults(get_time, name="get_time", description="Useful for getting the current time for a given location.")
all_tools = [weather_tool, time_tool]

agent = ReActAgent.from_tools(all_tools, llm=llm, verbose=True)

response = agent.chat("What is the time in New York, NY?")

print(response)


> Running step e4f14c6b-0703-45d9-9c5f-2e2eb879d7b2. Step input: What is the time in New York, NY?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: get_time
Action Input: {'location': 'New York, NY'}
Observation: The current time in New York, NY is 12:00 PM.
> Running step b5bd5e94-18aa-4ae3-894c-d5f4bd736613. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: The current time in New York, NY is 12:00 PM.
The current time in New York, NY is 12:00 PM.


In [7]:
## Create human in the loop agent with tools

from llama_index.core.agent import AgentRunner, ReActAgent
from llama_index.agent.openai import OpenAIAgentWorker, OpenAIAgent
from llama_index.agent.openai import OpenAIAgentWorker

agent_llm = OpenAI(model="gpt-4o-mini")
# agent_llm = OpenAI(model="gpt-4-1106-preview")

agent = ReActAgent.from_tools(
    all_tools, llm=agent_llm, verbose=True, max_iterations=20
)

def chat_repl(exit_when_done: bool = True):
    """Chat REPL.

    Args:
        exit_when_done(bool): if True, automatically exit when step is finished.
            Set to False if you want to keep going even if step is marked as finished by the agent.
            If False, you need to explicitly call "exit" to finalize a task execution.

    """
    task_message = None
    while task_message != "exit":
        task_message = input(">> Human: ")
        if task_message == "exit":
            break

        task = agent.create_task(task_message)

        response = None
        step_output = None
        message = None
        while message != "exit":
            if message is None or message == "":
                step_output = agent.run_step(task.task_id)
            else:
                step_output = agent.run_step(task.task_id, input=message)
            if exit_when_done and step_output.is_last:
                print(
                    ">> Task marked as finished by the agent, executing task execution."
                )
                break

            message = input(
                ">> Add feedback during step? (press enter/leave blank to continue, and type 'exit' to stop): "
            )
            if message == "exit":
                break

        if step_output is None:
            print(">> You haven't run the agent. Task is discarded.")
        elif not step_output.is_last:
            print(">> The agent hasn't finished yet. Task is discarded.")
        else:
            response = agent.finalize_response(task.task_id)
        print(f"Agent: {str(response)}")

In [8]:
## Test agent
chat_repl()

> Running step 23b6695e-1fff-494f-80ec-dde841a2c80f. Step input: What is the weather in Miami, FL?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: get_weather
Action Input: {'location': 'Miami, FL'}
Observation: The weather in Miami, FL is sunny.
> Running step da8ff92e-de85-4996-b257-b7d26c872023. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: The weather in Miami, FL is sunny.
>> Task marked as finished by the agent, executing task execution.
Agent: The weather in Miami, FL is sunny.
